# Exercise 4a
## 3 Red Cards Study

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score

### 3.1  Loading and Cleaning the Data

In [23]:
#load data with pandas
dataDyad=pd.read_csv('data/CrowdstormingDataJuly1st.csv')

In [3]:
#example
dataDyad.ix[[1200]].T

,1200
playerShort,toni-kroos
player,Toni Kroos
club,Bayern München
leagueCountry,Germany
birthday,04.01.1990
height,182
weight,78
position,Attacking Midfielder
games,1
victories,1


All features for Toni Kroos - ref 66 - dyad. The column <tt>games</tt> stands for the number of games in the player-referee dyad

We don't need all the features for our purposes. We can drop the features like <tt>player</tt>, <tt>club</tt>, <tt>height</tt>, <tt>yellowCards</tt> and <tt>photoID</tt>. <tt>yellowReds</tt> also gets dropped since it is not the same as a red card.

From <tt>Alpha_3</tt> to <tt>seExp</tt> the features are referee only. Since in the next step we will restructure the data set in a way that it is not dyad based anymore but playerbased. This step can be made because the question was to answer if "players with dark skin [are more likely to get red cards] than [...] players with light skin". Also in the next part we want to compute the fraction of games where one player gets a red card.

<span style ="color:green;font-weight:bold ">In the solution the following features were dropped:</span >

<span style ="color:green;font-weight:bold "><tt>'photoID', 'Alpha_3', 'yellowCards', 'meanIAT', 'nIAT', 'nIAT', 'seIAT', 'meanExp', 'nExp', 'seExp', 'refNum', 'refCountry'.</tt></span >

<span style ="color:green;font-weight:bold ">We also dropped the <tt>yellowReds</tt> because in our opinion this feature counts to how yellow cards are distributed but we are only interested in red cards.</span >

<span style ="color:green;font-weight:bold ">We also dropped the features <tt>victories, ties, defeats, goals</tt> which in retrospect was maybe not a very good idea as it may hold information to the player's degree of frustration. We could have summed those features just like we summed the <tt>games</tt> later on.</span >

In [4]:
dataDyad.drop(dataDyad.columns[[1,2,5,6,7,9,10,11,12,13,14,16,19,20,21,22,23,24,25,26,27]],1,inplace=True)

In [5]:
#now the data looks likt this:
dataDyad.ix[[1200]].T

,1200
playerShort,toni-kroos
leagueCountry,Germany
birthday,04.01.1990
games,1
redCards,0
rater1,0
rater2,0


In [6]:
#combine all instances of one player into one entry. Add all cards and games in the process
players=np.unique(dataDyad['playerShort']) #all the player names
data = dataDyad.groupby(dataDyad['playerShort']).agg({'playerShort':'first','leagueCountry':'first', 'birthday':'first',
                                                      'games': 'sum', 'redCards': 'sum','rater1':'first','rater2':'first'})

In [7]:
#transform birthday to age:
#season: 2012-2013, so at the end it was 2013
data['age'] = data['birthday'].apply(lambda x:2013- int(str(x)[-4:]))
#we can now drop the birthday column
data.drop('birthday',1,inplace=True)

In [8]:
data.ix[:19,['rater1','rater2']].T

playerShort,aaron-hughes,aaron-hunt,aaron-lennon,aaron-ramsey,abdelhamid-el-kaoutari,abdon-prats,abdou-dampha,abdou-traore_2,abdoul-camara,abdoulaye-diallo_2,abdoulaye-diallo_3,abdoulaye-keita_2,abdoulaye-sane,abdoulwhaid-sissoko,abdul-rahman-baba,abdul-razak,abel-aguilar,abel-khaled,abelaziz-barrada
rater1,0.25,0.00,0.25,0.0,0.25,NaN,NaN,0.75,NaN,0.75,NaN,0.75,NaN,1.0,0.75,1.0,0.50,NaN,0.0
rater2,0.00,0.25,0.25,0.0,0.25,NaN,NaN,0.75,NaN,1.00,NaN,1.00,NaN,1.0,1.00,1.0,0.25,NaN,0.0


In [9]:
np.mean(abs(data.ix[:,'rater1']-data.ix[:,'rater2'])),np.var(abs(data.ix[:,'rater1']-data.ix[:,'rater2']))

(0.06009463722397476, 0.011570022589537095)

I've looked at a small cut of the data set and the two raters do disagree occasionally by $0.25$. Bhe overall disagreement is relativley low with $0.060\pm0.012$

In [10]:
np.count_nonzero(np.isnan(data.ix[:,'rater1']))/len(data.ix[:,'rater1'])

0.22795908426692646

$\Rightarrow$ So around $22\%$ of the instances don't have a picture attached to them. All those instances don't help our case so we will drop them.

In [11]:
#drop dyads with no skin color rating
data.drop(data.index[np.where(np.isnan(data['rater1']))],inplace=True)

<span style ="color:green;font-weight:bold ">This was also done in the solution.</span >

In [12]:
leagues=np.unique(data.ix[:,'leagueCountry'])
print(leagues)

['England' 'France' 'Germany' 'Spain']


There are only the four leagues above in the data set. So a One-Hot could be the following:


|                     | England | France | Germany | Spain |
|---------------------|---------|--------|---------|-------|
| player from England | 1       | 0      | 0       | 0     |
| player from France  | 0       | 1      | 0       | 0     |
| player from Germany | 0       | 0      | 1       | 0     |
| player from Spain   | 0       | 0      | 0       | 1     |

We add to each player these columns and fill it with respect to his league Country

In [13]:
#create column for each country, set 1 if player is playing in that league and 0 if not
for country in leagues:
    data[country]=data['leagueCountry'].apply(lambda x:int(country==x))
#we can now drop the leagueCountry column
data.drop('leagueCountry',1,inplace=True)

<span style ="color:green;font-weight:bold ">This was done differently in the solution but the result should be the same</span >


In [14]:
#calculate labels...
labels=data['redCards']/data['games']
#...and drop all card related columns
data.drop(data.columns[:3],1,inplace=True)

In [15]:
#centralization of the data:
data=data-np.mean(data)
labels=np.array(labels-np.mean(labels))

In [16]:
#that's how the data looks now
data.head()

,rater1,rater2,age,England,France,Germany,Spain
playerShort,,,,,,,
aaron-hughes,-0.018612,-0.31041,6.899685,0.756467,-0.177918,-0.302208,-0.276341
aaron-hunt,-0.268612,-0.06041,-0.100315,-0.243533,-0.177918,0.697792,-0.276341
aaron-lennon,-0.018612,-0.06041,-1.100315,0.756467,-0.177918,-0.302208,-0.276341
aaron-ramsey,-0.268612,-0.31041,-4.100315,0.756467,-0.177918,-0.302208,-0.276341
abdelhamid-el-kaoutari,-0.018612,-0.06041,-4.100315,-0.243533,0.822082,-0.302208,-0.276341


### 3.2 Model Creation

#### Linear regression
$$\sum_{j=1}^DX_{ij}\beta_j=y_i,\ (\mathbb N \ni i\leq N)$$
$$\Rightarrow\mathbf X\mathbf \beta=\mathbf y,\ (\text{matrices})$$
$$\hat{\mathbf {\beta}}= \left(\mathbf{X^TX}\right)^{-1}\mathbf{X^T y}$$

In [17]:
class linearReg:       
    def train(self,x,y):
        '''
        Takes centralized data and finds the best beta (Dx1) vector
        x: (NxD) matrix with data
        y: (Nx1) vector with labels
        '''
        #centralize data:
        x=x-np.mean(x)
        y=y-np.mean(y)
        #calculate the moore penrose pseudo inverse:
        xPlus=np.linalg.pinv(x)
        self.beta=xPlus@y
        
    def predict(self,x):
        return self.beta@x

<span style ="color:green;font-weight:bold ">We didn't use linear_model from sklearn but instead used the pseudo inverse</span >

#### Regression tree

In [18]:
class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

In [19]:
#since our solution to the 4th exercise was not working we took the solution from moodle and adjusted it 
class RegressionTree(Tree):
    def __init__(self):
        super(RegressionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider 

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        #put root in stack
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:
                # Call 'make_regression_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                
                left, right = make_regression_split_node(node, np.arange(D)) #give all features since we only have a few
                                                       
                # put children in stack
                stack.append(left)
                stack.append(right)
            else:
                # Call 'make_regression_leaf_node()' to turn 'node' into a leaf node.
                make_regression_leaf_node(node)
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return leaf.response 

<span style ="color:green;font-weight:bold ">We had to give all features instead of just $\lfloor \sqrt D \rfloor$ random features because we dropped to many.</span >


In [20]:
def make_regression_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = 1e100
    j_min, t_min = 0, 0
    for j in feature_indices:
        # remove duplicate features
        dj = np.sort(np.unique(node.data[:,j]))
        # compute candidate thresholds in the middle between consecutive feature values
        tj = 0.5 * (dj[1:] + dj[:-1]) 
        # each candidate threshold we need to compute squared error of the resulting children node
        for t in tj:
            left_indices = node.data[:,j] <= t
            nl = np.sum(left_indices)
            ll = node.labels[left_indices]
            el = np.sum(np.square(ll-np.mean(ll)))
            nr = n - nl
            lr = node.labels[node.data[:,j] > t]
            er = np.sum(np.square(lr-np.mean(lr)))
            # choose the the best threshold that minimizes sum of the squared error.
            if el + er < e_min:
                e_min = el + er
                j_min = j
                t_min = t

    
    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above
    left.data = node.data[node.data[:,j_min] <= t_min, :]
    left.labels = node.labels[node.data[:,j_min] <= t_min]
    right.data = node.data[node.data[:,j_min] > t_min, :]
    right.labels = node.labels[node.data[:,j_min] > t_min]

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right    

In [21]:
def make_regression_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    node.N = node.data.shape[0]
    node.response = np.sum(node.labels) / node.N

<span style ="color:green;font-weight:bold "><tt>make_regression_split_node</tt> and <tt>make_regression_leaf_node</tt> are equivalent</span >

In [22]:
#forest
class RegForest:
    def __init__(self,n=10):
        # create n instances of Regression tree 
        self.trees = [RegressionTree() for i in range(n)]
    
    def train(self, data, target, n_min=20):
        # train all trees
        for tree in self.trees:
            tree.train(data, target, n_min)
            
    def predict(self, x):
        # return the mean for the RegressionTree that maximizes p(x | y)
        return np.mean([tree.predict(x) for tree in self.trees])

In [84]:
#crossvalidation
def cross_validation(methode,X,Y,num_sample=10,out=False):
    """
    Measure the correct accuracy with cross validation
    methode: class Forest or linearReg or similar
    """
    mean_rate = np.zeros(num_sample)
    for i in range(num_sample):
        x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.33, random_state=None)
        f=methode()
        f.train(x_train,y_train)
        predicted_labels=[]
        for j in range(len(x_test)):
            predicted_labels.append(f.predict(x_test[j]))
        mean_rate[i] = np.mean(np.square(predicted_labels - y_test))
    mean,var=np.mean(mean_rate), np.std(mean_rate)
    if out:
        print(methode.__name__,"Mean squared test error: %e +/- %e"%(mean,var))
    return mean


In [312]:
X=np.array(data)
unscatterdMeanLin=cross_validation(linearReg,X,labels,out=True)

linearReg Mean squared test error: 5.254971e-05 +/- 1.639992e-05


In [313]:
#test models:
unscatterdMeanTree=cross_validation(RegForest,X,labels,out=True)

RegForest Mean squared test error: 5.404153e-05 +/- 1.206830e-05


The squared error for both models is similar.

### 3.3 Answering the Research Question

In [85]:
#permutation test
def shuffleSkinColors(data):
    newData=data.copy()
    seed=np.random.randint(2e9)
    np.random.seed(seed)
    np.random.shuffle(newData['rater1'].values)
    #change the 'rater2' values in the same way as the 'rater1' values
    np.random.seed(seed)
    np.random.shuffle(newData['rater2'].values)
    return newData

In [314]:
count=0
tree,lin=0,0
for i in range(19):
    #permute skin color features:
    X=np.array(shuffleSkinColors(data))
    meanLin=cross_validation(linearReg,X,labels)
    meanTree=cross_validation(RegForest,X,labels,num_sample=10)
    #if the error is larger than the one of the original dataset we add one to the count
    if meanLin>unscatterdMeanLin and meanTree>unscatterdMeanTree:
         count+=1
    elif meanLin>unscatterdMeanLin:
        tree+=1 #Regression Tree says error is smaller than before
    elif meanTree>unscatterdMeanTree:
        lin+=1 #Linear Regression says error is smaller than before
    print('%.2f'%(100*(i+1)/19),'% done',end='\r')

In [318]:
if count==19:
    print('Skin color bias (19/19 for color bias)')
else:
    print('No skin color bias (%i/19 for color bias)'%count)
print('%i times did the Regression Tree conclude the error is smaller than before and the Linear Regression disagrees'%tree)
print('%i times did the Linear Regression conclude the error is smaller than before and the Regression Tree disagrees'%lin)
print('%i times they both agreed that the errors became smaller'%(19-count-tree-lin))

No skin color bias (14/19 for color bias)
0 times did the Regression Tree conclude the error is smaller than before and the Linear Regression disagrees
5 times did the Linear Regression conclude the error is smaller than before and the Regression Tree disagrees
0 times they both agreed that the errors became smaller


The result of the permutation test is that the skin color is _not_ dependent on whether a player receives a red card. In all the 5 instances that voted against a color bias the Regression Forest dissagreed with the Linear Regression method. Since the Linear Regression is a fairly simple model it could be at its limits. Another possibility is that the permutation did not change many skin color values because we only have 5 distinct skin color values, so the possibility that a player gets the same or a similar skin color is quite high, which if a bias existed could be detected by our models. 

### other data preperation

In [113]:
#load data again
dataDyad=pd.read_csv('data/CrowdstormingDataJuly1st.csv')
#drop other features than before: keep height and weight. They might influence the play and therefor the frequency of a red card
#also don't drop the position
dataDyad.drop(dataDyad.columns[[1,2,9,10,11,12,13,14,16,19,20,21,22,23,24,25,26,27]],1,inplace=True)

In [114]:
#combine again all instances of one player into one entry. Add all cards and games in the process
players=np.unique(dataDyad['playerShort']) #all the player names
data = dataDyad.groupby(dataDyad['playerShort']).agg({'playerShort':'first','leagueCountry':'first', 'birthday':'first',
                                                      'height':'first','weight':'first','position':'first',
                                                      'games': 'sum', 'redCards': 'sum','rater1':'first','rater2':'first'})

In [115]:
#transform birthday to age:
#season: 2012-2013, so at the end it was 2013
data['age'] = data['birthday'].apply(lambda x:2013- int(str(x)[-4:]))
#we can now drop the birthday column
data.drop('birthday',1,inplace=True)

In [116]:
#drop dyads with no skin color rating
data.drop(data.index[np.where(np.isnan(data['rater1']))],inplace=True)
#the new features height and weight also have nan entries
data.drop(data.index[np.where(np.isnan(data['height']))],inplace=True)
data.drop(data.index[np.where(np.isnan(data['weight']))],inplace=True)

In [117]:
#create column for each country, set 1 if player is playing in that league and 0 if not
for country in leagues:
    data[country]=data['leagueCountry'].apply(lambda x:int(country==x))
#we can now drop the leagueCountry column
data.drop('leagueCountry',1,inplace=True)

In [118]:
#introduce a one hot encoding for the position
positions=np.array(dataDyad.ix[:,'position'])
positions[pd.isnull(positions)]='None' #first get rid of the float
positions=np.unique(positions) #now we have all of them

#create column for each position, set 1 if player is playing in that position and 0 if not
for pos in positions:
    data[pos]=data['position'].apply(lambda x:int(pos==x))
#we can now drop the positions column
data.drop('position',1,inplace=True)

In [119]:
#calculate labels...
labels=data['redCards']/data['games']
#...and drop all card related columns
data.drop(['playerShort','games','redCards'],1,inplace=True)

In [120]:
#that's how the data looks now
data.head()

,height,weight,rater1,rater2,age,England,France,Germany,Spain,Attacking Midfielder,...,Center Midfielder,Defensive Midfielder,Goalkeeper,Left Fullback,Left Midfielder,Left Winger,None,Right Fullback,Right Midfielder,Right Winger
playerShort,,,,,,,,,,,,,,,,,,,,,
aaron-hughes,182.0,71.0,0.25,0.00,34,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaron-hunt,183.0,73.0,0.00,0.25,27,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
aaron-lennon,165.0,63.0,0.25,0.25,26,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
aaron-ramsey,178.0,76.0,0.00,0.00,23,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
abdelhamid-el-kaoutari,180.0,73.0,0.25,0.25,23,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [121]:
#centralization of the data:
data=data-np.mean(data)
X=np.array(data)
labels=np.array(labels-np.mean(labels))

#### Retry with new set of data

In [122]:
unscatterdMeanLinNew=cross_validation(linearReg,X,labels,out=True)

linearReg Mean squared test error: 4.974823e-05 +/- 1.609284e-05


In [123]:
#test models:
unscatterdMeanTreeNew=cross_validation(RegForest,X,labels,out=True)

RegForest Mean squared test error: 7.043198e-05 +/- 1.871536e-05


In [124]:
count=0
tree,lin=0,0
for i in range(19):
    #permute skin color features:
    X=np.array(shuffleSkinColors(data))
    meanLin=cross_validation(linearReg,X,labels)
    meanTree=cross_validation(RegForest,X,labels,num_sample=10)
    #if the error is larger than the one of the original dataset we add one to the count
    if meanLin>unscatterdMeanLinNew and meanTree>unscatterdMeanTreeNew:
         count+=1
    elif meanLin>unscatterdMeanLinNew:
        tree+=1 #Regression Tree says error is smaller than before
    elif meanTree>unscatterdMeanTreeNew:
        lin+=1 #Linear Regression says error is smaller than before
    print('%.2f'%(100*(i+1)/19),'% done',end='\r')

In [125]:
if count==19:
    print('Skin color bias (19/19 for color bias)')
else:
    print('No skin color bias (%i/19 for color bias)'%count)
print('%i times did the Regression Tree conclude the error is smaller than before and the Linear Regression disagrees'%tree)
print('%i times did the Linear Regression conclude the error is smaller than before and the Regression Tree disagrees'%lin)
print('%i times they both agreed that the errors became smaller'%(19-count-tree-lin))

No skin color bias (10/19 for color bias)
4 times did the Regression Tree conclude the error is smaller than before and the Linear Regression disagrees
4 times did the Linear Regression conclude the error is smaller than before and the Regression Tree disagrees
1 times they both agreed that the errors became smaller


The grade of bias is lower than befor. That could be because we now included more features that influence the play style of a certain player. For example a more heavy player can foul another player more easily than a light player because of the momentum each player can transfer to the opponent. This is only a guess since I am no expert. It also could be that the lighter player needs to play more aggressivley to make an impact, leading to a higher rate of red cards. It carries information we didn't have befor so we don't need to rely on the skin color feature if not necessary.

This dataset could not get an opposing result but it showed nonetheless that the data preperation is an important and influential step of statistical analysis.